# Google API i geokodowania

[Piotr Migdał](http://migdal.wikidot.com/), DELab UW

Będziemy geokodować adresy przy pomocy  [The Google Geocoding API](https://developers.google.com/maps/documentation/geocoding/).

### Słowniczek

* [Geokodowanie](https://pl.wikipedia.org/wiki/Geokodowanie) - ustalanie współrzędnych geograficznych na podstawie innych danych geograficznych (np. kodu pocztowego, ulicy i miasta). 
* [API](https://pl.wikipedia.org/wiki/Application_Programming_Interface) (Application Programming Interface) - ogólne słowo sposób komunikacji z programem, zwłaszcza: zdalnym
* [JSON](https://pl.wikipedia.org/wiki/JSON) (JavaScript Object Notation) - tekstowy sposób zapisywania danych, który pozwala zapisać różne dane, nie tylko tabele (jak CSV)
* [XML](https://en.wikipedia.org/wiki/XML) (Extensible Markup Language) 

### Zapytania

* https://maps.googleapis.com/maps/api/geocode/json?address=ul.+Dobra+56/66,+Warszawa
    * dobra lokalizacja? sprawdzić możemy np. tak:
    https://www.google.com/maps/@52.2423917,21.0244245,16z
* https://maps.googleapis.com/maps/api/timezone/json?location=39.6034810,-119.6822510&timestamp=1331161200
* https://maps.googleapis.com/maps/api/elevation/json?locations=39.7391536,-104.9847034
* https://maps.googleapis.com/maps/api/directions/json?origin=Warszawa&destination=Kraków

Spróbuj:

* Zastanowić co się w nich pyta, i co oznacza odpowiedź.
* Zadać inne pytania.
* Poprosić o odpowiedź w formacie XML (spójrz na podobieństwa i różnice).

### Ograniczenia

Wersja darmowa Google API pozwala na 2500 zapytań w ciągu 24h, i nie więcej niż 5 na sekundę.

### Inne przykładowe API

* http://api.worldbank.org/countries?format=json
* https://en.wikipedia.org/w/api.php?action=query&list=backlinks&bltitle=Almond&bllimit=500&blfilterredir=nonredirects&format=json
* https://api.stackexchange.com/2.2/tags/python/related?site=stackoverflow

### Python

Surowa forma pytań i odpowiedzi jest celowa - ułatwia przetwarzanie przez komputer (dla niego łatwije wysłać "json?location=39.6034810,-119.6822510&timestamp=1331161200" niż wyklikać na stronie internetowej).


In [3]:
# biblioteka do zapytań 
import requests

In [4]:
r = requests.get("https://maps.googleapis.com/maps/api/elevation/json?locations=39.7391536,-104.9847034")

In [ ]:
# zapytanie może się skończyć w różny sposób - np. n

In [9]:
# surowa treść odpowiedzi
r.content

b'{\n   "results" : [\n      {\n         "elevation" : 1608.637939453125,\n         "location" : {\n            "lat" : 39.7391536,\n            "lng" : -104.9847034\n         },\n         "resolution" : 4.771975994110107\n      }\n   ],\n   "status" : "OK"\n}\n'

In [12]:
# wczytanie JSONa
odpowiedz = r.json()
odpowiedz

{'results': [{'elevation': 1608.637939453125,
   'location': {'lat': 39.7391536, 'lng': -104.9847034},
   'resolution': 4.771975994110107}],
 'status': 'OK'}

In [13]:
odpowiedz['status']

'OK'

In [14]:
odpowiedz['results']

[{'elevation': 1608.637939453125,
  'location': {'lat': 39.7391536, 'lng': -104.9847034},
  'resolution': 4.771975994110107}]

In [17]:
odpowiedz['results'][0]['location']['lng']

-104.9847034

## Przekazywanie parametrów

Typowe zapytania mają parematry. Np.

* https://maps.googleapis.com/maps/api/directions/json?origin=Warszawa&destination=Kraków

składa się z

* głównego adresu `https://maps.googleapis.com/maps/api/directions/json`,
* parametru `origin` równemu `Warszawa`
* i parametru `destination` równemy `Kraków`.

Zapytania mają postać `glowny_adres?parametr1=wartosc1&parametr2=wartosc2&parametr3=wartosc3` itd. Zwykle kolejność parametrów nie ma znaczenia.

Do zapytań można "ręcznie" tworzyć pełen adres, lub generować parametry z pythonowych słowników (druga opcja jest zwkle dużo przyjemniejsza).

In [19]:
r = requests.get("https://maps.googleapis.com/maps/api/directions/json",
                 params={"origin": "Warszawa", "destination": "Kraków"})

In [29]:
# tak wygląda wygenerowany adres url
r.url

'https://maps.googleapis.com/maps/api/directions/json?destination=Krak%C3%B3w&origin=Warszawa'

In [35]:
odpowiedz = r.json()
odpowiedz

{'routes': [{'bounds': {'northeast': {'lat': 52.22968119999999,
     'lng': 21.1494693},
    'southwest': {'lat': 50.0646454, 'lng': 19.9449779}},
   'copyrights': 'Map data ©2015 Google',
   'legs': [{'distance': {'text': '296 km', 'value': 295559},
     'duration': {'text': '4 hours 1 min', 'value': 14456},
     'end_address': 'Kraków, Poland',
     'end_location': {'lat': 50.0646454, 'lng': 19.9449779},
     'start_address': 'Warsaw, Poland',
     'start_location': {'lat': 52.22968119999999, 'lng': 21.0121891},
     'steps': [{'distance': {'text': '0.1 km', 'value': 110},
       'duration': {'text': '1 min', 'value': 24},
       'end_location': {'lat': 52.2299527, 'lng': 21.0112515},
       'html_instructions': 'Head <b>north</b> toward <b>Aleje Jerozolimskie/DW631</b>',
       'polyline': {'points': 'obx}He}f_CGEO@E?G@E@A?C@A?A@A@C@EBABGFEHEFAL?B?P@N@L@H@F?B@HBHBHBHDJFD'},
       'start_location': {'lat': 52.22968119999999, 'lng': 21.0121891},
       'travel_mode': 'DRIVING'},
    

Dla ścieżki odpowiedź jest długa, bo podaje listę możliwych tras, a każda trasa składa się z punktów. Jak to eksporować?

In [36]:
odpowiedz.keys()

dict_keys(['status', 'routes'])

In [37]:
odpowiedz["status"]

'OK'

In [40]:
len(odpowiedz["routes"])

1

In [42]:
odpowiedz["routes"][0].keys()

dict_keys(['legs', 'overview_polyline', 'waypoint_order', 'warnings', 'copyrights', 'summary', 'bounds'])

In [47]:
len(odpowiedz["routes"][0]["legs"][0]["steps"])

19

In [50]:
odpowiedz["routes"][0]["legs"][0]["distance"]

{'text': '296 km', 'value': 295559}

In [48]:
odpowiedz["routes"][0]["legs"][0]["steps"][5]

{'distance': {'text': '2.0 km', 'value': 2021},
 'duration': {'text': '2 mins', 'value': 97},
 'end_location': {'lat': 52.16852429999999, 'lng': 20.92702},
 'html_instructions': 'Keep <b>left</b> at the fork to continue toward <b>aleja Krakowska/DK7/DK8/E67/E77</b>',
 'maneuver': 'fork-left',
 'polyline': {'points': '{xm}H_ts~BjEjA~F`Bl@RlEhBjBt@b@Vz@dAL\\FVBZ?X?XCVARGTCJEJILONOLMHQFK@U@UESGKKMMKQGSG]COCaBd@cC`@sB`@sBhAwFLk@Ji@p@_D`@qBf@eCbAaF`CaLX{AjAwFd@{B`@mBzAqHh@oCd@sBFWP}@lA_G'},
 'start_location': {'lat': 52.1769385, 'lng': 20.9134358},
 'travel_mode': 'DRIVING'}

## Co jeszcze możemy z nimi zrobić?

Oczywiście - zapisać, odczytać.

In [51]:
import json

In [53]:
# tak zapisujemy plik JSON
json.dump(odpowiedz, open("../dane/waw_krk.json", "w"), indent=2)
# teraz warto go otworzy

Warto zobaczyć jak wygląda w edytorze tekstu, np. [Sublime Text](http://www.sublimetext.com/) (płatny, ale można długo próbować) lub [Notepad++](https://notepad-plus-plus.org/) (darmowy, Windows). Do mniejszych plików (tak do 2MB) jest [Atom](https://atom.io/). Jak chcesz głęboką wodę, to można spróbować Vim lub Emacs.

In [55]:
# a tak wczytujemy
zaladowany = json.load(open("../dane/waw_krk.json"))

In [56]:
zaladowany

{'routes': [{'bounds': {'northeast': {'lat': 52.22968119999999,
     'lng': 21.1494693},
    'southwest': {'lat': 50.0646454, 'lng': 19.9449779}},
   'copyrights': 'Map data ©2015 Google',
   'legs': [{'distance': {'text': '296 km', 'value': 295559},
     'duration': {'text': '4 hours 1 min', 'value': 14456},
     'end_address': 'Kraków, Poland',
     'end_location': {'lat': 50.0646454, 'lng': 19.9449779},
     'start_address': 'Warsaw, Poland',
     'start_location': {'lat': 52.22968119999999, 'lng': 21.0121891},
     'steps': [{'distance': {'text': '0.1 km', 'value': 110},
       'duration': {'text': '1 min', 'value': 24},
       'end_location': {'lat': 52.2299527, 'lng': 21.0112515},
       'html_instructions': 'Head <b>north</b> toward <b>Aleje Jerozolimskie/DW631</b>',
       'polyline': {'points': 'obx}He}f_CGEO@E?G@E@A?C@A?A@A@C@EBABGFEHEFAL?B?P@N@L@H@F?B@HBHBHBHDJFD'},
       'start_location': {'lat': 52.22968119999999, 'lng': 21.0121891},
       'travel_mode': 'DRIVING'},
    

In [59]:
# wracając do lokalizacji
lokalizacja_adres = "https://maps.googleapis.com/maps/api/geocode/json"

zlokalizowane = requests.get(lokalizacja_adres, params={"address": "Dobra 56, Warszawa"}).json()
zlokalizowane

{'results': [{'address_components': [{'long_name': '56',
     'short_name': '56',
     'types': ['street_number']},
    {'long_name': 'Dobra', 'short_name': 'Dobra', 'types': ['route']},
    {'long_name': 'Śródmieście',
     'short_name': 'Śródmieście',
     'types': ['sublocality_level_1', 'sublocality', 'political']},
    {'long_name': 'Warszawa',
     'short_name': 'Warszawa',
     'types': ['locality', 'political']},
    {'long_name': 'Warszawa',
     'short_name': 'Warszawa',
     'types': ['administrative_area_level_2', 'political']},
    {'long_name': 'mazowieckie',
     'short_name': 'mazowieckie',
     'types': ['administrative_area_level_1', 'political']},
    {'long_name': 'Poland',
     'short_name': 'PL',
     'types': ['country', 'political']},
    {'long_name': '00-384', 'short_name': '00-384', 'types': ['postal_code']}],
   'formatted_address': 'Dobra 56, 00-384 Warszawa, Poland',
   'geometry': {'bounds': {'northeast': {'lat': 52.2422762, 'lng': 21.0242115},
     'sout

Do rzeczy! Zlokalizujmy szkoły w Warszawie.

Dane pochodzą z Zespołu Pomiaru Dydaktycznego Instytutu Badań Edukacyjnych, przez [pakiet R zozlak/ZDP](https://github.com/zozlak/ZPD), następnie przetworzony w ramach [badania wyników matur w DELab](https://github.com/stared/delab-matury).

In [63]:
import pandas as pd

In [64]:
licea = pd.read_csv("../dane/licea_warszawskie.csv")

In [69]:
len(licea)

165

In [65]:
licea.head()

,id_szkoly,nazwa_szkoly,adres,miejscowosc,kod_pocztowy,dzielnica
0,25302,Liceum Ogólnokształcące dla Dorosłych nr 48 MGM,Obrońców Tobruku 40,Warszawa,01-494,Bemowo
1,25300,Sportowe Liceum Ogólnokształcące nr 67 MGM,Obrońców Tobruku 40,Warszawa,01-494,Bemowo
2,25298,LXXVIII Liceum Ogólnokształcące im. Marii Pawl...,Anieli Krzywoń 3,Warszawa,01-391,Bemowo
3,25499,Liceum Ogólnokształcące Niepubliczne nr 29 im....,Powązkowska 90,Warszawa,01-728,Bemowo
4,25303,CV Liceum Ogólnokształcące im. Zbigniewa Herberta,Vincenta van Gogha 1,Warszawa,03-188,Białołęka


In [68]:
licea["dzielnica"].value_counts()

Śródmieście       33
Mokotów           23
Praga-Południe    17
Wola              16
Bielany           13
Ursynów           12
Praga-Północ       9
Ochota             8
Żoliborz           7
Wawer              7
Targówek           6
Bemowo             4
Włochy             2
Wilanów            2
Wesoła             2
Białołęka          2
Ursus              1
Rembertów          1
dtype: int64

### Zadanie

Dla wybranej przez siebie dzielnicy Warszawy dodać kolumny `szerokosc` i `dlugosc` zgodnie z geolokalizacją szkoły.